# Shader Generation

This book will examine how to set up MaterialX for code generation. This book covers the <a href="https://github.com/AcademySoftwareFoundation/MaterialX/blob/main/python/Scripts/generateshader.py" target="_blank">
generateshader.py</a> script provided as part of the core distribution.

Topics covered:
1. Shading language 'target's
2. Module / library organization
3. Setting up generators and generation contexts 
5. Real world units and color management 
6. Discovering "renderable" items, and generating code
7. Extracting source code

Details behind code generation, generation options, introspection / reflection,
and input binding is covered as part of rendering.  

Background on code generation can be found <a href="https://github.com/AcademySoftwareFoundation/MaterialX/blob/main/documents/DeveloperGuide/ShaderGeneration.md" target ="_blank">here.</a>

## Code Generation Modules / Libraries

For the Python distribution, each code generator resides in separate module in the MaterialX package. 

The name of each module is of the form:
```
   PyMaterialXGen<target>
``` 
where target is the code generation target written in camel-case. 

All target names start with `gen` and then the shading language name:
```
   gen<target>
``` 

For example, the target for the OSL shading language is `genosl`, with the module's postfix string being `GenOsl`.
The variants for GLSL include: `genglsl` and `genessl`, which reside in a single module with postfix string `GenGlsl`.

The C++ library equivalent to the module is named:
```
   MaterialXGen<target>
```
Basically the same as the Python module but without the `Py` prefix.

The module `PyMaterialxShader` contains the base support for all code generators. In the code below this module as well as modules for all targets are imported.

In [370]:
import sys, os, subprocess
import MaterialX as mx
import MaterialX.PyMaterialXGenShader as mx_gen_shader
import MaterialX.PyMaterialXGenGlsl as mx_gen_glsl
import MaterialX.PyMaterialXGenOsl as mx_gen_osl
import MaterialX.PyMaterialXGenMdl as mx_gen_mdl


## Setup

The basic setup requires that a document is created, the standard libraries are loaded, and the document containing the elements to generate code for to be present.

Additional modules can be imported to support functionality such as code validation.

### Code Validation

For `GLSL`, `ESSL`, and `Vulkan` languages <a href="https://github.com/KhronosGroup/glslang" target="_blank">glslangValidator</a>  can be used for syntax and compilation validation. It is installed using `vcpkg` and is run as part of the CI process. For OSL and MDL: `olsc` and `mdlc` compilers are used respectively.

The `generateshader.py` script supports passing in a external program as an argument. The source code passed to this program for validation.

The utility function from that script has been extracted out and is included below as an example.

In [371]:

def validateCode(sourceCodeFile, codevalidator, codevalidatorArgs):
    if codevalidator:
        cmd = codevalidator + ' ' + sourceCodeFile 
        if codevalidatorArgs:
            cmd += ' ' + codevalidatorArgs
        print('----- Run Validator: '+ cmd)
        try:
            output = subprocess.check_output(cmd, stderr=subprocess.STDOUT)
            result = output.decode(encoding='utf-8')
        except subprocess.CalledProcessError as out:                                                                                                   
            return (out.output.decode(encoding='utf-8'))
    return ""


### Exception Handling

In the following code, a document is first created and then a sample file which defines a "marble" material is read in.

Note that MaterialX throws exceptions when encountering errors instead of keeping track of status code.
There are some specific exceptions which provide additional information beyond the regular exception information.

It is always prudent to catch exceptions including checking of custom exceptions 
provided for file I/O, code generation and rendering.

In this example a "file missing" exception may be returned if the file cannot be read.

 The possible `Exception` types are defined in the <a href="https://materialx.org/docs/api/class_exception.html" target="_blank">API documentation</a>. In Python, the exception name is the same as the C++ class name.

In [372]:
# Read in MaterialX file
#
inputFilename = 'standard_surface_marble_solid.mtlx'
doc = mx.createDocument()
try:
    mx.readFromXmlFile(doc, inputFilename)    
    
    valid, msg = doc.validate()
    if not valid:
        raise mx.Exception('Document is invalid')

    print('Read in valid file "'"%s"'" for code generation.' % inputFilename)

except mx.ExceptionMissing as err:
    print('File %s could not be loaded: "' % inputFilename, err, '"')
except mx.Exception as err:
    print('File %s fail to load properly: "' % inputFilename, err, '"')


Read in valid file "standard_surface_marble_solid.mtlx" for code generation.


### Implementations

The standard library includes both definitions as well as implementations for each shading language. 
The `stdlib`, `stdlib`, and `bxdf` folders contain definitions and any corresponding node graph and
source code implementations.

The sub-folders starting with `gen` contain per-target source code implementations.

<pre>
├───bxdf
│   ├───lama
│   └───translation
├───lights
│   └───genglsl
├───pbrlib
│   ├───genglsl  <-- e.g. target 'genglsl's implementations reside here
│   ├───genmdl
│   └───genosl
├───stdlib
│   ├───genglsl
│   ├───genmdl
│   └───genosl
└───targets
</pre>

As this code is required for code generation to occur,  the standard `libraries` folder must be read in. 

Implementations are of the type <a href="https://materialx.org/docs/api/class_implementation.html" target="_blank">`Implementation`</a>.

In [373]:
# Load in standard libraries
stdlib = mx.createDocument()
searchPath = mx.FileSearchPath(os.path.dirname(inputFilename))
libraryFolders = []
libraryFolders.append("libraries")
try:
    mx.loadLibraries(libraryFolders, searchPath, stdlib)
    doc.importLibrary(stdlib)
    print('Standard library definitions and implementations loaded.')
except err:
    print('Standard library definitions and implementation failed to load: "', err, '"')
    sys.exit(-1)


Standard library definitions and implementations loaded.


The `getImplementations()` API is used to get a list of `Implementation` references. Even though the total number
of implementations seem large, only the source code for a specific generator are used at any given time.

In [374]:
# Get list of all implementations
implmentations = doc.getImplementations()
if implmentations:
    print('Read in %d implementations' % len(implmentations))

Read in 1746 implementations


## Implementation "Targets"

Every non-nodegraph implementation must specify a `target` that it supports. 

A `target` name is used to identify shading languages and their variants. The naming convention is:
```
  gen<language name>
```
These are represented as a <a href="https://materialx.org/docs/api/class_target_def.html" target="_blank">`TargetDef`</a>. 
The target identifiers are loaded in as part of the standard library, and these can be queried by
looking for elements of category `targetdef`. For convenience, a list of available targets can be retrieved from a 
document using the `getTargetDefs()` API.

However, at time of writing, this is missing from the Python API. Thus a simple utility function is provided here.

In [375]:
# The targetdef type and support API does not currently exist so cannot be used.
#doc.getTargetDefs()
#doc.getChildOfType(mx.TargetDef)

# Utility that basically does what doc.getTargetDefs() does.
# Matching the category can be used in lieu to testing for the class type.
def getTargetDefs(doc):
    targets = []
    for element in doc.getChildren():
        if element.getCategory() == 'targetdef':
            targets.append(element.getName())
    return targets

foundTargets = getTargetDefs(doc)
for target in foundTargets:
    implcount = 0
    # Find out how many implementations we have 
    for impl in implmentations:
        testtarget = target
        if target == 'essl':
            testtarget = 'genglsl'
        if impl.getTarget() == testtarget:
            implcount = implcount + 1
    print('Found target identifier:', target, 'with', implcount, 'source implementations.')      

Found target identifier: essl with 580 source implementations.
Found target identifier: genglsl with 580 source implementations.
Found target identifier: genmdl with 585 source implementations.
Found target identifier: genosl with 577 source implementations.


## Code Generators

### Generator Creation

Every code generator must have a `target` identifier to indicate which shading langauge / variant it supports.
A language version can be used to distinguish a variant if appropriate (e.g. ESSL is distinguishable this way)

It is recommended that all new generators have a unique target name. 

Currently there is no "registry" for generators by target so the user must know before hand which generators exist and
go through all generators to find one with the appropriate `target` to use.

Targets themselves can be "inherited" which is reflected in the inheritance hierarchy for generators.
For example the `essl` (ESSL) target inherits from the `genglsl` (GLSL) target as does the corresponding generators. 
Inheritance is generally used to specialize code generation to handle shading language variations. 

For a list of generators and their derivations see documentation for the base class <a href="https://materialx.org/docs/api/class_shader_generator.html" target="_blank">ShaderGenerator</a>

<img src="https://kwokcb.github.io/MaterialX_Learn/documents/images/ShaderGenerator_inheritance.png" style="width:50%"></img>

Note that Vulkan has the same target as `genglsl`, but has it's own generator.

Integrations are free to create custom generators. Some notable existing generators include those used to support USD HDStorm, VEX, and Arnold OSL.

Any such generator can be instantiated and use the same generation process as described here.

For this example, we will show how all the the generators can be created, but will only produce OSL code via an `OslShaderGenerator` generator. This can be found in the `PyMaterialXGenOsl` Python submodule and corresponding `MaterialXGenOsl` library in C++.  

In [376]:

# Create all generators
generators = []
generators.append(mx_gen_osl.OslShaderGenerator.create())
generators.append(mx_gen_mdl.MdlShaderGenerator.create())
generators.append(mx_gen_glsl.EsslShaderGenerator.create())
generators.append(mx_gen_glsl.VkShaderGenerator.create())

# Create a dictionary based on target identifier
generatordict = {}
for gen in generators:
    generatordict[gen.getTarget()] = gen

# Choose generator to use based on target identifier
language = 'mdl'
target = 'genosl'
if language == 'osl':
    target = 'genosl'
elif language == 'mdl':
    target = 'genmdl'
elif language == 'essl':
    target = 'essl'
elif language in ['essl', 'glsl', 'vulkan']:
    target = 'genglsl'

test_language = 'essl'
test_shadergen = generatordict[test_language]
print('Find code generator for target:', test_shadergen.getTarget(), ' for language:', test_language, ". Version:", test_shadergen.getVersion())

shadergen = generatordict[target]
print('Use code generator for target:', shadergen.getTarget(), ' for language: ', language)

Find code generator for target: essl  for language: essl . Version: 300 es
Use code generator for target: genmdl  for language:  mdl


### Generator Contexts

A "generation context" is required to be created for each generator instance. This is represented by a 
<a href="https://materialx.org/docs/api/class_gen_context.html" target="_blank">`GenContext`</a> structure. 

This context provides a number of settings and options to be used for code generation. 

For simplicity, we will only point out the minimal requirements. This includes providing a search path to where source code implementations can be found. Any number of paths can be added using the `registerSourceCodeSearchPath()` function, on the context. The search order is first to last path added.

Adding the path to the `libraries` folder is sufficient to find the source code for standard library definitions found in sub-folders. If the user has custom definitions in other locations, the root of those locations should be added. 

In [377]:
 # Create a context for a generator
context = mx_gen_shader.GenContext(shadergen)

# Register a path to where implmentations can be found.
context.registerSourceCodeSearchPath(searchPath)


### Color Management

Color management is used to ensure that input colors are interpreted properly via shader code.

A "color management system" cab be created and specified to be used by a shader generator. 
During code generation, additional logic is emitted into the shader source code via the system.

Usage of such as system during code generation is optional, as some renderers perform color management on input values
and images before binding them to shading code.

Color management systems need to be derived from the base API interface <a href="https://materialx.org/docs/api/class_color_management_system.html" target="_blank">`ColorManagementSystem`</a>). A "default" system is provided
as part of the MaterialX distribution.

It is necessary to indicate which `target` shading language code when instantiating the color management system. Naturally specifying a non-matching target will inject incompatible code.

The setup steps are:

1. Create the system. In this example the "default" system is created with the `target` being specified at creation time.
2. Setting where the library of definitions exists for the system. In this case the main document which contains the standard library is specified.
3. Setting the color management system on the generator. If it is not set or cleared then no color management will occur during code generation.

In [378]:
# Create default CMS
cms = mx_gen_shader.DefaultColorManagementSystem.create(shadergen.getTarget())  
# Indicate to the CMS where definitions can be found
cms.loadLibrary(doc)
# Indicate to the code generator to use this CMS
shadergen.setColorManagementSystem(cms)

cms = shadergen.getColorManagementSystem()
if cms:
    print('Set up CMS: %s for target: %s' 
          % (cms.getName(), shadergen.getTarget()))

Set up CMS: default_cms for target: genmdl


### Real World Units

To handle real-world unit specifiers a "unit system" should be instantiated and associated with the generator.
The API interface is a <a href="https://materialx.org/docs/api/class_unit_system.html" target="_blank">UnitSystem</a> which definitions. 

By default a unit system does not know how to perform any conversions. This is provided by a 
<a href="https://materialx.org/docs/api/class_unit_converter_registry.html" target="_blank">`UnitConverterRegistry`</a> which contains a list of convertors. 

Currently MaterialX supports convertors for converting linear units: distance, and angle.
The corresponding API interface is <a href="https://materialx.org/docs/api/class_linear_unit_converter.html" target="_blank">`LinearUnitConverter`</a>.


In [379]:
# Create unit registry
registry = mx.UnitConverterRegistry.create()
if registry:
    # Get distance and angle unit type definitions and create a linear converter for each
    distanceTypeDef = doc.getUnitTypeDef('distance')
    if distanceTypeDef:
        registry.addUnitConverter(distanceTypeDef, mx.LinearUnitConverter.create(distanceTypeDef))
    angleTypeDef = doc.getUnitTypeDef('angle')
    if angleTypeDef:
        registry.addUnitConverter(angleTypeDef, mx.LinearUnitConverter.create(angleTypeDef))
    print('Created unit converter registry')

Created unit converter registry


As with a color management system the location of implementations and the registry need to be set on a unit system. 
The unit system can then be set on the generator.

In [380]:
# Create unit system, set where definitions come from, and
# set up what registry to use
unitsystem = mx_gen_shader.UnitSystem.create(shadergen.getTarget())
unitsystem.loadLibrary(stdlib)
unitsystem.setUnitConverterRegistry(registry)

if unitsystem:
    print('Set unit system on code generator')
    shadergen.setUnitSystem(unitsystem)


Set unit system on code generator


This sets up how to perform unit conversions, but does not specify what unis the scene geometry is using.
This can be specified as in an "options" structure found on the  context.

The API interface is: <a href="https://materialx.org/docs/api/class_gen_options.html"  target="_blank">GenOptions</a>.

In [381]:
# Set the target scene unit to be `meter` on the context options
genoptions = context.getOptions()
genoptions.targetDistanceUnit = 'meter'

## Finding Elements to Render

There are a few utilities which are included to find elements which are "renderable":

1. The <a href="https://materialx.org/docs/api/_material_x_gen_shader_2_util_8h.html" target="_blank">findRenderableElement()</a> utility can be used in general to find these. 
2. Another possible utility is to find only material nodes using `getMaterialNodes()` or 
2. Shader nodes by looking for nodes of type `SURFACE_SHADER_TYPE_STRING` in a document.

For this example, the first "renderable" found is used.



In [382]:

# Look for renderable nodes
nodes = mx_gen_shader.findRenderableElements(doc, False)
print('Found: %d renderables' % len(nodes))
if not nodes:
    nodes = doc.getMaterialNodes()
    if not nodes:
        nodes = doc.getNodesOfType(mx.SURFACE_SHADER_TYPE_STRING)

node = None 
if nodes:
    node = nodes[0]
    print('Found node to render: ', node.getName())


Found: 1 renderables
Found node to render:  Marble_3D


## Generating Code

After all of this setup, code can now be generated.
1. First a `createValidName()` utility is called to ensure that the shader name produced is valid. 
2. Then the generator's <a href="https://materialx.org/docs/api/class_shader_generator.html" target="_blank">generate()</a> interface is called with this name, the "renderable" element, and the generation context. Note that derived classes override `generate()` to perform custom generation.

Upon success a new <a href="https://materialx.org/docs/api/class_shader.html" target="_blank">Shader</a> instance is created. Note that this is a special interface used to keep track of an entire shader. This instance can be inspected to extract information required for rendering.

In [383]:
shader = None
nodeName = node.getName() if node else ''
if nodeName:
    shaderName = mx.createValidName(nodeName)
    try:
        shader = shadergen.generate(shaderName, node, context)
    except err:
        print('Shader generation errors:', err)

if shader:
    print('Succeeded in generating code for shader "%s" code from node "%s"' % (shaderName, nodeName)) 
else:
    print('Failed to generate code for shader "%s" code from node "%s"' % (shaderName, nodeName)) 

Succeeded in generating code for shader "Marble_3D" code from node "Marble_3D"


### Generated Code

For hardware languages like GLSL, vertex, and pixel shader code is generated. OSL and MDL only produce
pixel shader code. To complete this example the pixel shader code is queried from the Shader and
shown below.

Code can be queried via the <a href="https://materialx.org/docs/api/class_shader.html" target="_blank">getSourceCode()</a> interface with an argument indicating which code to return. The code returned can be directly compiled and used by a renderer.

It is at this point in the `generateshader.py` script that validation is performed. (This will not be shown here.) 

In [384]:
pixelSource = ''
vertexSource = ''
if shader:
    errors = ''

    # Use extension of .vert and .frag as it's type is
    # recognized by glslangValidator
    if language in ['glsl', 'essl', 'vulkan']:
        vertexSource = shader.getSourceCode(mx_gen_shader.VERTEX_STAGE)
        display_markdown('### Vertex Source Code' , raw=True)
        display_markdown('------------------', raw=True)
        display_markdown('```ccc {' + vertexSource + '}```', raw=True)

    pixelSource = shader.getSourceCode(mx_gen_shader.PIXEL_STAGE)
    display_markdown('### Pixel Source Code' , raw=True)
    display_markdown('------------------', raw=True)
    display_markdown('```cpp {' + pixelSource + ' }```', raw=True)


### Pixel Source Code

------------------

```cpp {mdl 1.6;

using mx = materialx;
import ::df::*;
import ::base::*;
import ::math::*;
import ::state::*;
import ::anno::*;
import ::tex::*;
import ::mx::swizzle::*;
import ::mx::cm::*;
using ::mx::core import *;
using ::mx::stdlib import *;
using ::mx::pbrlib import *;
using ::mx::sampling import *;

material NG_standard_surface_surfaceshader_100
(
    float base = 0.8,
    color base_color = color(1, 1, 1),
    float diffuse_roughness = 0,
    float metalness = 0,
    float specular = 1,
    color specular_color = color(1, 1, 1),
    float specular_roughness = 0.2,
    uniform float specular_IOR = 1.5,
    float specular_anisotropy = 0,
    float specular_rotation = 0,
    float transmission = 0,
    color transmission_color = color(1, 1, 1),
    float transmission_depth = 0,
    color transmission_scatter = color(0, 0, 0),
    float transmission_scatter_anisotropy = 0,
    float transmission_dispersion = 0,
    float transmission_extra_roughness = 0,
    float subsurface = 0,
    color subsurface_color = color(1, 1, 1),
    color subsurface_radius = color(1, 1, 1),
    float subsurface_scale = 1,
    float subsurface_anisotropy = 0,
    float sheen = 0,
    color sheen_color = color(1, 1, 1),
    float sheen_roughness = 0.3,
    float coat = 0,
    color coat_color = color(1, 1, 1),
    float coat_roughness = 0.1,
    float coat_anisotropy = 0,
    float coat_rotation = 0,
    uniform float coat_IOR = 1.5,
    float3 coat_normal = state::transform_normal(state::coordinate_internal, state::coordinate_world, state::normal()),
    float coat_affect_color = 0,
    float coat_affect_roughness = 0,
    float thin_film_thickness = 0,
    float thin_film_IOR = 1.5,
    float emission = 0,
    color emission_color = color(1, 1, 1),
    color opacity = color(1, 1, 1),
    bool thin_walled = false,
    float3 normal = state::transform_normal(state::coordinate_internal, state::coordinate_world, state::normal()),
    float3 tangent = state::transform_vector(state::coordinate_internal, state::coordinate_world, state::texture_tangent_u(0))
)
 = let
{
    float2 coat_roughness_vector_out = mx::pbrlib::mx_roughness_anisotropy(mxp_roughness:coat_roughness, mxp_anisotropy:coat_anisotropy);
    float coat_tangent_rotate_degree_out = coat_rotation * 360;
    color metal_reflectivity_out = base_color * base;
    color metal_edgecolor_out = specular_color * specular;
    float coat_affect_roughness_multiply1_out = coat_affect_roughness * coat;
    float tangent_rotate_degree_out = specular_rotation * 360;
    float transmission_roughness_add_out = specular_roughness + transmission_extra_roughness;
    color subsurface_color_nonnegative_out = math::max(subsurface_color, 0);
    float coat_clamped_out = math::clamp(coat, 0, 1);
    float3 subsurface_radius_vector_out = float3(float3(subsurface_radius).x, float3(subsurface_radius).y, float3(subsurface_radius).z);
    float subsurface_selector_out = float(thin_walled);
    color base_color_nonnegative_out = math::max(base_color, 0);
    color coat_attenuation_out = math::lerp(color(1, 1, 1), coat_color, coat);
    color emission_weight_out = emission_color * emission;
    color coat_emission_attenuation_out = math::lerp(color(1, 1, 1), coat_color, coat);
    color opacity_luminance_out = mx::stdlib::mx_luminance_color3(opacity);
    float3 coat_tangent_rotate_out = mx::stdlib::mx_rotate3d_vector3(mxp_in:tangent, mxp_amount:coat_tangent_rotate_degree_out, mxp_axis:coat_normal);
    mx::pbrlib::mx_artistic_ior__result artistic_ior_result = mx::pbrlib::mx_artistic_ior(mxp_reflectivity:metal_reflectivity_out, mxp_edge_color:metal_edgecolor_out);
    float coat_affect_roughness_multiply2_out = coat_affect_roughness_multiply1_out * coat_roughness;
    float3 tangent_rotate_out = mx::stdlib::mx_rotate3d_vector3(mxp_in:tangent, mxp_amount:tangent_rotate_degree_out, mxp_axis:normal);
    float transmission_roughness_clamped_out = math::clamp(transmission_roughness_add_out, 0, 1);
    float coat_gamma_multiply_out = coat_clamped_out * coat_affect_color;
    float3 subsurface_radius_scaled_out = subsurface_radius_vector_out * subsurface_scale;
    color emission_weight_attenuated_out = emission_weight_out * coat_emission_attenuation_out;
    float3 coat_tangent_rotate_normalize_out = math::normalize(coat_tangent_rotate_out);
    float coat_affected_roughness_out = math::lerp(specular_roughness, 1, coat_affect_roughness_multiply2_out);
    float3 tangent_rotate_normalize_out = math::normalize(tangent_rotate_out);
    float coat_affected_transmission_roughness_out = math::lerp(transmission_roughness_clamped_out, 1, coat_affect_roughness_multiply2_out);
    float coat_gamma_out = coat_gamma_multiply_out + 1;
    float3 coat_tangent_out = mx::stdlib::mx_ifgreater_vector3(coat_anisotropy, 0, coat_tangent_rotate_normalize_out, tangent);
    float2 main_roughness_out = mx::pbrlib::mx_roughness_anisotropy(mxp_roughness:coat_affected_roughness_out, mxp_anisotropy:specular_anisotropy);
    float3 main_tangent_out = mx::stdlib::mx_ifgreater_vector3(specular_anisotropy, 0, tangent_rotate_normalize_out, tangent);
    float2 transmission_roughness_out = mx::pbrlib::mx_roughness_anisotropy(mxp_roughness:coat_affected_transmission_roughness_out, mxp_anisotropy:specular_anisotropy);
    color coat_affected_subsurface_color_out = math::pow(subsurface_color_nonnegative_out, coat_gamma_out);
    color coat_affected_diffuse_color_out = math::pow(base_color_nonnegative_out, coat_gamma_out);
    material metal_bsdf_out = mx::pbrlib::mx_conductor_bsdf(mxp_weight:1, mxp_ior:artistic_ior_result.mxp_ior, mxp_extinction:artistic_ior_result.mxp_extinction, mxp_roughness:main_roughness_out, mxp_normal:normal, mxp_tangent:main_tangent_out);
    material transmission_bsdf_out = mx::pbrlib::mx_dielectric_bsdf(mxp_weight:1, mxp_tint:transmission_color, mxp_ior:specular_IOR, mxp_roughness:transmission_roughness_out, mxp_normal:normal, mxp_tangent:main_tangent_out, mxp_scatter_mode:mx_scatter_mode_T, mxp_base:material());
    material translucent_bsdf_out = mx::pbrlib::mx_translucent_bsdf(mxp_weight:1, mxp_color:coat_affected_subsurface_color_out, mxp_normal:normal);
    material subsurface_bsdf_out = mx::pbrlib::mx_subsurface_bsdf(mxp_weight:1, mxp_color:coat_affected_subsurface_color_out, mxp_radius:subsurface_radius_scaled_out, mxp_anisotropy:subsurface_anisotropy, mxp_normal:normal);
    material selected_subsurface_bsdf_out = mx::pbrlib::mx_mix_bsdf(mxp_fg:translucent_bsdf_out, mxp_bg:subsurface_bsdf_out, mxp_mix:subsurface_selector_out);
    material diffuse_bsdf_out = mx::pbrlib::mx_oren_nayar_diffuse_bsdf(mxp_weight:base, mxp_color:coat_affected_diffuse_color_out, mxp_roughness:diffuse_roughness, mxp_normal:normal);
    material subsurface_mix_out = mx::pbrlib::mx_mix_bsdf(mxp_fg:selected_subsurface_bsdf_out, mxp_bg:diffuse_bsdf_out, mxp_mix:subsurface);
    material sheen_layer_out = mx::pbrlib::mx_sheen_bsdf(mxp_weight:sheen, mxp_color:sheen_color, mxp_roughness:sheen_roughness, mxp_normal:normal, mxp_base:subsurface_mix_out);
    material transmission_mix_out = mx::pbrlib::mx_mix_bsdf(mxp_fg:transmission_bsdf_out, mxp_bg:sheen_layer_out, mxp_mix:transmission);
    material specular_layer_out = mx::pbrlib::mx_dielectric_bsdf(mxp_weight:specular, mxp_tint:specular_color, mxp_ior:specular_IOR, mxp_roughness:main_roughness_out, mxp_normal:normal, mxp_tangent:main_tangent_out, mxp_scatter_mode:mx_scatter_mode_R, mxp_base:transmission_mix_out);
    material metalness_mix_out = mx::pbrlib::mx_mix_bsdf(mxp_fg:metal_bsdf_out, mxp_bg:specular_layer_out, mxp_mix:metalness);
    material thin_film_layer_out = mx::pbrlib::mx_thin_film_bsdf(mxp_thickness:thin_film_thickness, mxp_ior:thin_film_IOR, mxp_base:metalness_mix_out);
    material thin_film_layer_attenuated_out = mx::pbrlib::mx_multiply_bsdf_color3(mxp_in1:thin_film_layer_out, mxp_in2:coat_attenuation_out);
    material coat_layer_out = mx::pbrlib::mx_dielectric_bsdf(mxp_weight:coat, mxp_tint:color(1, 1, 1), mxp_ior:coat_IOR, mxp_roughness:coat_roughness_vector_out, mxp_normal:coat_normal, mxp_tangent:coat_tangent_out, mxp_scatter_mode:mx_scatter_mode_R, mxp_base:thin_film_layer_attenuated_out);
    material emission_edf_out = mx::pbrlib::mx_uniform_edf(mxp_color:emission_weight_attenuated_out);
    material shader_constructor_out = mx::pbrlib::mx_surface(coat_layer_out, emission_edf_out, float3(opacity_luminance_out).x, specular_IOR);
}
in material(shader_constructor_out);

export material Marble_3D
(
    float3 displacementshader = float3(0.0),
    uniform mx_coordinatespace_type geomprop_Nworld_space = mx_coordinatespace_type_world,
    uniform mx_coordinatespace_type geomprop_Tworld_space = mx_coordinatespace_type_world,
    uniform int geomprop_Tworld_index = 0,
    uniform mx_coordinatespace_type obj_pos_space = mx_coordinatespace_type_object,
    float3 add_xyz_in2 = float3(1, 1, 1),
    float scale_pos_in2 = 4,
    float scale_xyz_in2 = 6,
    float noise_amplitude = 1,
    int noise_octaves = 3,
    float noise_lacunarity = 2,
    float noise_diminish = 0.5,
    float scale_noise_in2 = 3,
    float scale_in2 = 0.5,
    float bias_in2 = 0.5,
    float power_in2 = 3,
    color color_mix_fg = color(0.1, 0.1, 0.3),
    color color_mix_bg = color(0.8, 0.8, 0.8),
    float SR_marble1_base = 1,
    float SR_marble1_diffuse_roughness = 0,
    float SR_marble1_metalness = 0,
    float SR_marble1_specular = 1,
    color SR_marble1_specular_color = color(1, 1, 1),
    float SR_marble1_specular_roughness = 0.1,
    uniform float SR_marble1_specular_IOR = 1.5,
    float SR_marble1_specular_anisotropy = 0,
    float SR_marble1_specular_rotation = 0,
    float SR_marble1_transmission = 0,
    color SR_marble1_transmission_color = color(1, 1, 1),
    float SR_marble1_transmission_depth = 0,
    color SR_marble1_transmission_scatter = color(0, 0, 0),
    float SR_marble1_transmission_scatter_anisotropy = 0,
    float SR_marble1_transmission_dispersion = 0,
    float SR_marble1_transmission_extra_roughness = 0,
    float SR_marble1_subsurface = 0.4,
    color SR_marble1_subsurface_radius = color(1, 1, 1),
    float SR_marble1_subsurface_scale = 1,
    float SR_marble1_subsurface_anisotropy = 0,
    float SR_marble1_sheen = 0,
    color SR_marble1_sheen_color = color(1, 1, 1),
    float SR_marble1_sheen_roughness = 0.3,
    float SR_marble1_coat = 0,
    color SR_marble1_coat_color = color(1, 1, 1),
    float SR_marble1_coat_roughness = 0.1,
    float SR_marble1_coat_anisotropy = 0,
    float SR_marble1_coat_rotation = 0,
    uniform float SR_marble1_coat_IOR = 1.5,
    float SR_marble1_coat_affect_color = 0,
    float SR_marble1_coat_affect_roughness = 0,
    float SR_marble1_thin_film_thickness = 0,
    float SR_marble1_thin_film_IOR = 1.5,
    float SR_marble1_emission = 0,
    color SR_marble1_emission_color = color(1, 1, 1),
    color SR_marble1_opacity = color(1, 1, 1),
    bool SR_marble1_thin_walled = false
)
= let
{
    float3 geomprop_Nworld_out = mx::stdlib::mx_normal_vector3(mxp_space:geomprop_Nworld_space);
    float3 geomprop_Tworld_out = mx::stdlib::mx_tangent_vector3(mxp_space:geomprop_Tworld_space, mxp_index:geomprop_Tworld_index);
    float3 obj_pos_out = mx::stdlib::mx_position_vector3(mxp_space:obj_pos_space);
    float add_xyz_out = math::dot(obj_pos_out, add_xyz_in2);
    float3 scale_pos_out = obj_pos_out * scale_pos_in2;
    float scale_xyz_out = add_xyz_out * scale_xyz_in2;
    float noise_out = mx::stdlib::mx_fractal3d_float(mxp_amplitude:noise_amplitude, mxp_octaves:noise_octaves, mxp_lacunarity:noise_lacunarity, mxp_diminish:noise_diminish, mxp_position:scale_pos_out);
    float scale_noise_out = noise_out * scale_noise_in2;
    float sum_out = scale_xyz_out + scale_noise_out;
    float sin_out = math::sin(sum_out);
    float scale_out = sin_out * scale_in2;
    float bias_out = scale_out + bias_in2;
    float power_out = math::pow(bias_out, power_in2);
    color color_mix_out = math::lerp(color_mix_bg, color_mix_fg, power_out);
    material SR_marble1_out = NG_standard_surface_surfaceshader_100(SR_marble1_base, color_mix_out, SR_marble1_diffuse_roughness, SR_marble1_metalness, SR_marble1_specular, SR_marble1_specular_color, SR_marble1_specular_roughness, SR_marble1_specular_IOR, SR_marble1_specular_anisotropy, SR_marble1_specular_rotation, SR_marble1_transmission, SR_marble1_transmission_color, SR_marble1_transmission_depth, SR_marble1_transmission_scatter, SR_marble1_transmission_scatter_anisotropy, SR_marble1_transmission_dispersion, SR_marble1_transmission_extra_roughness, SR_marble1_subsurface, color_mix_out, SR_marble1_subsurface_radius, SR_marble1_subsurface_scale, SR_marble1_subsurface_anisotropy, SR_marble1_sheen, SR_marble1_sheen_color, SR_marble1_sheen_roughness, SR_marble1_coat, SR_marble1_coat_color, SR_marble1_coat_roughness, SR_marble1_coat_anisotropy, SR_marble1_coat_rotation, SR_marble1_coat_IOR, geomprop_Nworld_out, SR_marble1_coat_affect_color, SR_marble1_coat_affect_roughness, SR_marble1_thin_film_thickness, SR_marble1_thin_film_IOR, SR_marble1_emission, SR_marble1_emission_color, SR_marble1_opacity, SR_marble1_thin_walled, geomprop_Nworld_out, geomprop_Tworld_out);
    material Marble_3D_out = mx::stdlib::mx_surfacematerial(SR_marble1_out, displacementshader);
    material finalOutput__ = Marble_3D_out;
}
in material(finalOutput__);
 }```